In [11]:
# Load, model using Kimball principles, and validate

import pandas as pd

# Load CSVs
artists = pd.read_csv("artists.csv")
releases = pd.read_csv("releases.csv")
streams = pd.read_csv("streams.csv")

# Standardize column names
artists.columns = artists.columns.str.strip().str.lower()
releases.columns = releases.columns.str.strip().str.lower()
streams.columns = streams.columns.str.strip().str.lower()

# Create dim_artists
dim_artists = artists.drop_duplicates(subset=['artist_id']).copy()
dim_artists['artist_sk'] = dim_artists.reset_index().index + 1

# Create dim_releases
dim_releases = releases.drop_duplicates(subset=['release_id']).copy()
dim_releases['release_sk'] = dim_releases.reset_index().index + 1

# Create surrogate key maps
artist_sk_map = dim_artists.set_index('artist_id')['artist_sk'].to_dict()
release_sk_map = dim_releases.set_index('release_id')['release_sk'].to_dict()

# Enrich streams with artist_id by joining with releases
streams_enriched = streams.merge(releases[['release_id', 'artist_id']], on='release_id', how='left')

# Build fact_streams with surrogate keys
fact_streams = streams_enriched.copy()
fact_streams['artist_sk'] = fact_streams['artist_id'].map(artist_sk_map)
fact_streams['release_sk'] = fact_streams['release_id'].map(release_sk_map)
fact_streams['stream_date'] = pd.to_datetime(fact_streams['stream_date'])

# Final columns for fact table
fact_streams = fact_streams[['artist_sk', 'release_sk', 'stream_date', 'platform', 'region', 'streams']]
fact_streams = fact_streams.rename(columns={'streams': 'stream_count'})

# Data quality checks
print(" Data Quality Checks:")
print("- dim_artists PK unique:", dim_artists['artist_sk'].is_unique)
print("- dim_releases PK unique:", dim_releases['release_sk'].is_unique)
print("- Nulls in fact_streams:\n", fact_streams.isnull().sum())

# Display top rows
print("\n dim_artists:")
display(dim_artists.head())

print("\n dim_releases:")
display(dim_releases.head())

print("\n fact_streams:")
display(fact_streams.head())

 Data Quality Checks:
- dim_artists PK unique: True
- dim_releases PK unique: True
- Nulls in fact_streams:
 artist_sk       0
release_sk      0
stream_date     0
platform        0
region          0
stream_count    0
dtype: int64

 dim_artists:


artist_id         artist_name        country   join_date  artist_sk
0          1    Thomas Rodriguez  Liechtenstein  2021-07-01          1
1          2           Ann Young         Jordan  2025-04-26          2
2          3  Matthew Stephenson        Bolivia  2025-03-24          3
3          4           Juan Kemp        Eritrea  2022-04-27          4
4          5           Jon Small         Mexico  2023-11-09          5


 dim_releases:


release_id  artist_id                                release_title  \
0           1          7             Stand-alone upward-trending core   
1           2          3      Multi-layered upward-trending algorithm   
2           3         10  Stand-alone zero administration methodology   
3           4          5          Object-based motivating open system   
4           5          1       Fully-configurable optimizing matrices   

  release_date       genre  release_sk  
0   2022-01-26        Jazz           1  
1   2023-09-30  Electronic           2  
2   2022-09-16        Jazz           3  
3   2024-05-04        Rock           4  
4   2021-05-21        Jazz           5


 fact_streams:


artist_sk  release_sk stream_date      platform                 region  \
0          7          17  2024-12-30   Apple Music  Palestinian Territory   
1         10           3  2025-03-20  Amazon Music              Indonesia   
2          6           8  2025-03-19       Spotify                  Qatar   
3          4           9  2024-07-05  Amazon Music           Burkina Faso   
4          8          11  2024-12-05       Spotify               Kiribati   

   stream_count  
0          9893  
1          4907  
2          1742  
3          3973  
4          6949